# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [2]:
%load_ext dotenv
%dotenv

In [2]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [3]:
import os
from glob import glob

# Check the value of PRICE_DATA
price_data_dir = os.getenv("PRICE_DATA")
print("PRICE_DATA directory:", price_data_dir)

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [4]:
import dask.dataframe as dd

# Load the data from Parquet files
dd_price_data = dd.read_parquet('C:\\Users\\Lisa\\production\\05_src\\data\\prices')

# Define a function to process each ticker separately
def process_ticker(data):
    sample_data = dd_price_data.head(100)  # Load a small sample to test
    processed_sample = process_ticker(sample_data)
    print(processed_sample)
   
    # Create lagged variables for Close and Adj_Close
    data['Close_lag_1'] = data['Close'].shift(1)
    data['Adj_Close_lag'] = data['Adj_Close'].shift(1)
    
    # Calculate returns based on Adj_Close
    data['returns'] = (data['Adj_Close'] / data['Adj_Close_lag']) - 1
    
    # Calculate hi_lo_range 
    data['hi_lo_range'] = data['High'] - data['Low']
    
    return data

# Define the metadata for the output DataFrame
meta = {
    'Date': 'datetime64[ns]',
    'ticker': 'object',
    'Close': 'float64',
    'Adj_Close': 'float64',
    'Close_lag_1': 'float64',
    'Adj_Close_lag': 'float64',
    'returns': 'float64',
    'High': 'float64',
    'Low': 'float64',
    'hi_lo_range': 'float64'
}

# Group by 'ticker' and apply the function
dd_feat = dd_price_data.groupby('ticker').apply(process_ticker, meta=meta)

# Show sample of the result 
dd_feat.compute()



+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [25]:
#Convert to pandas DataFrame
df_pandas = df_dask.compute()

# Add a rolling average return calculation with a 10-day window
# Replace 'return_column' with the actual column name you want to calculate the rolling mean on
df_pandas['rolling_avg_return'] = df_pandas['return_column'].rolling(window=10).mean()

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
No - we could have done the moving average in Dask

+ Would it have been better to do it in Dask? Why?
Pandas loads the entire DataFrame into memory, whereas in Dask it is done in chunks


(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.